In [19]:
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

import os
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from sklearn.model_selection import train_test_split
from sagemaker.tensorflow import TensorFlow

Couldn't call 'get_role' to get Role ARN from role name AmazonSageMaker-ExecutionRole-20210118T135727 to get Role path.
Assuming role was created in SageMaker AWS console, as the name contains `AmazonSageMaker-ExecutionRole`. Defaulting to Role ARN with service-role in path. If this Role ARN is incorrect, please add IAM read permissions to your role or supply the Role Arn directly.


# Train Model

In [37]:
ds_data = np.load('dataset3/data.npy', allow_pickle=True)
ds_labels = np.load('dataset3/labels.npy', allow_pickle=True)
x_train, x_val, y_train, y_val = train_test_split(ds_data, ds_labels, train_size=0.75)

os.makedirs("./data", exist_ok = True)

np.savez('./data/training', image=x_train, label=y_train)
np.savez('./data/validation', image=x_val, label=y_val)

prefix = 'ml-screen-reader'
training_input_path   = sess.upload_data('data/training.npz', key_prefix=prefix+'/training')
validation_input_path = sess.upload_data('data/validation.npz', key_prefix=prefix+'/validation')

print(training_input_path)
print(validation_input_path)

tf_estimator = TensorFlow(entry_point='model_builders/dnn.py', 
                          role=role,
                          instance_count=1, 
                          instance_type='ml.m5.xlarge',
                          framework_version='2.3', 
                          py_version='py37',
                          script_mode=True,
                          hyperparameters={
                              'epochs': 20,
                              'batch-size': 256,
                              'learning-rate': 0.01}
                         )
# tf_estimator = TensorFlow(entry_point='model_builders/dnn.py', 
#                           role=role,
#                           instance_count=1, 
#                           instance_type='local',
#                           framework_version='1.12', 
#                           py_version='py3',
#                           script_mode=True,
#                           hyperparameters={'epochs': 1}
#                          )

tf_estimator.fit({'training': training_input_path, 'validation': validation_input_path})

s3://sagemaker-us-east-2-435269695145/ml-screen-reader/training/training.npz
s3://sagemaker-us-east-2-435269695145/ml-screen-reader/validation/validation.npz
2021-03-26 23:07:08 Starting - Starting the training job...
2021-03-26 23:07:32 Starting - Launching requested ML instancesProfilerReport-1616800028: InProgress
......
2021-03-26 23:08:32 Starting - Preparing the instances for training...
2021-03-26 23:08:58 Downloading - Downloading input data...
2021-03-26 23:09:37 Training - Training image download completed. Training in progress..2021-03-26 23:09:38.954571: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-03-26 23:09:38.958485: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-03-26 23:09:39.131499: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the Sa

# Load Model

In [ ]:
MODEL_DIRECTORY = 'models'
MODEL = ''
VERSION = ''
tf_estimator = tf.saved_model.load(os.path.join(MODEL_DIRECTORY, MODEL, VERSION))

# Deploy Model

In [38]:
tf_endpoint_name = 'ml-screen-reader-endpoint'
tf_predictor = tf_estimator.deploy(initial_instance_count=1,
                                  instance_type='ml.m4.xlarge',
                                  endpoint_name=tf_endpoint_name)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!

# Update Model

In [30]:
sess.update_endpoint(endpoint_name=tf_endpoint_name, endpoint_config_name=tf_endpoint_name)

ClientError: An error occurred (ValidationException) when calling the UpdateEndpoint operation: Cannot update endpoint "ml-screen-reader-endpoint" with the currently in use endpoint configuration "ml-screen-reader-endpoint".

# Delete Endpoint

In [ ]:
tf_endpoint_name = 'ml-screen-reader-endpoint'
sess.delete_endpoint(endpoint_name=tf_endpoint_name)